Installing any dependencies required

In [1]:
!pip install nltk
!pip install gensim
!pip install wordcloud
!pip install kaggle
!pip install PyPDF2
!pip install datasets
!pip install torch
!pip install transformers
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.4 MB/s eta 0:00:00
2023-09-19 09:23:52.218926: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow wi

Importing the necessary libraries

In [2]:
from google.colab import files
import os
import pandas as pd
import numpy as np
import torch
import glob
import io
from pdfminer.high_level import extract_text

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

import gensim
import re
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.metrics import classification_report, confusion_matrix

from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

Giving access to kaggle to load the resume dataset

In [3]:
# Upload your Kaggle API token (kaggle.json)
uploaded = files.upload()

# Move the Kaggle API token to the appropriate directory
if not os.path.exists('/root/.kaggle'):
    os.makedirs('/root/.kaggle')

!mv kaggle.json /root/.kaggle/

# Change permissions on the token
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


Downloading the required dataset

In [4]:
# Download the Kaggle dataset
!kaggle datasets download snehaanbhawal/resume-dataset

# Unzip the downloaded dataset
!unzip resume-dataset.zip

 69% 43.0M/62.5M [00:00<00:00, 106MB/s]
100% 62.5M/62.5M [00:00<00:00, 129MB/s]
Archive:  resume-dataset.zip
  inflating: Resume/Resume.csv       
  inflating: data/data/ACCOUNTANT/10554236.pdf  
  inflating: data/data/ACCOUNTANT/10674770.pdf  
  inflating: data/data/ACCOUNTANT/11163645.pdf  
  inflating: data/data/ACCOUNTANT/11759079.pdf  
  inflating: data/data/ACCOUNTANT/12065211.pdf  
  inflating: data/data/ACCOUNTANT/12202337.pdf  
  inflating: data/data/ACCOUNTANT/12338274.pdf  
  inflating: data/data/ACCOUNTANT/12442909.pdf  
  inflating: data/data/ACCOUNTANT/12780508.pdf  
  inflating: data/data/ACCOUNTANT/12802330.pdf  
  inflating: data/data/ACCOUNTANT/13072019.pdf  
  inflating: data/data/ACCOUNTANT/13130984.pdf  
  inflating: data/data/ACCOUNTANT/13294301.pdf  
  inflating: data/data/ACCOUNTANT/13491889.pdf  
  inflating: data/data/ACCOUNTANT/13701259.pdf  
  inflating: data/data/ACCOUNTANT/14055988.pdf  
  inflating: data/data/ACCOUNTANT/14126433.pdf  
  inflating: data/da

#Resume Mathing using the pdf provided in dataset

In [40]:
root_directory = "data/data/"

# List of all subdirectories
subdirectories = [d for d in os.listdir(root_directory) if os.path.isdir(os.path.join(root_directory, d))]

pdf_pattern = "*.pdf"

resumes = []

# Loop through each subdirectory
for subdirectory in subdirectories:
    pdf_directory = os.path.join(root_directory, subdirectory)

    pdf_files = glob.glob(os.path.join(pdf_directory, pdf_pattern))

    # Loop through each PDF file in the subdirectory
    for pdf_file in pdf_files:
        rel_path = os.path.relpath(pdf_file, root_directory)

        with open(pdf_file, "rb") as pdf:
            pdf_bytesio = io.BytesIO(pdf.read())

            # Extract text from the file-like object
            text = extract_text(pdf_bytesio)

        # Append the resumes
        resumes.append(text)

In [22]:
df1 = pd.DataFrame(resumes)

Loading the job Descriptions

In [23]:
dataset = load_dataset("jacob-hugging-face/job-descriptions")

job_description = dataset['train']['job_description'][:15]
df_cv = df1[0]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading DistilBERT tokenizer and model

In [24]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Tokenize and obtain embeddings for CV details and job descriptions
cv_embeddings = []
job_embeddings = []

Finding embeddings

In [25]:
for cv_text in df_cv:
    inputs = tokenizer(cv_text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    cv_embeddings.append(embeddings)

for job_text in job_description:
    inputs = tokenizer(job_text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    job_embeddings.append(embeddings)

In [27]:
(np.array(cv_embeddings)).shape

(353, 1, 768)

In [29]:
# Calculate cosine similarity between CV details and job descriptions
matching_scores = cosine_similarity(np.array(cv_embeddings).reshape(353, 768), np.array(job_embeddings).reshape(15, 768))

# Threshold for matching
threshold = 0.9

# Match job descriptions to CV details
top_matches = {}

for j, job_desc in enumerate(job_description):
  if j<15:
    scores = matching_scores[:, j]
    matched_cvs = [(i, score) for i, score in enumerate(scores) if score > threshold]
    matched_cvs.sort(key=lambda x: x[1], reverse=True)
    top_matches[j] = matched_cvs[:5]  # Get the top 5 matched CVs for each job description

# Print the matching results
for job_idx, top_cvs in top_matches.items():
    job_desc = job_description[job_idx]
    print(f"Job Description {job_idx + 1}: ")
    print("Top Matched CVs:")
    for cv_idx, score in top_cvs:
        cv_text = df_cv[cv_idx]
        print(f"- CV Detail {cv_idx + 1} (Cosine Similarity Score: {score:.5f})")
    print("-" * 50)


Job Description 1: 
Top Matched CVs:
- CV Detail 222 (Cosine Similarity Score: 0.94160)
- CV Detail 187 (Cosine Similarity Score: 0.94131)
- CV Detail 80 (Cosine Similarity Score: 0.94105)
- CV Detail 229 (Cosine Similarity Score: 0.93878)
- CV Detail 245 (Cosine Similarity Score: 0.93813)
--------------------------------------------------
Job Description 2: 
Top Matched CVs:
- CV Detail 292 (Cosine Similarity Score: 0.91157)
- CV Detail 93 (Cosine Similarity Score: 0.91115)
- CV Detail 80 (Cosine Similarity Score: 0.91058)
- CV Detail 63 (Cosine Similarity Score: 0.90926)
- CV Detail 55 (Cosine Similarity Score: 0.90349)
--------------------------------------------------
Job Description 3: 
Top Matched CVs:
- CV Detail 291 (Cosine Similarity Score: 0.94660)
- CV Detail 229 (Cosine Similarity Score: 0.94621)
- CV Detail 61 (Cosine Similarity Score: 0.94342)
- CV Detail 292 (Cosine Similarity Score: 0.94300)
- CV Detail 245 (Cosine Similarity Score: 0.94149)
----------------------------

#Resume Mathing using the csv provided in dataset

In [31]:
# Read the CSV file
df = pd.read_csv('Resume/Resume.csv')
df = df[:150]

In [32]:
df.drop(columns = ['Resume_html'], inplace = True)

In [33]:
nltk.download('punkt')
nltk.download("stopwords")

def preprocess(txt):
    # convert all characters in the string to lower case
    txt = txt.lower()
    # remove non-english characters, punctuation and numbers
    txt = re.sub('[^a-zA-Z]', ' ', txt)
    # tokenize word
    txt = nltk.tokenize.word_tokenize(txt)

    return ' '.join(txt)

stop_words = stopwords.words('english')

def remove_stop_words (text):
  result = []
  for token in gensim.utils.simple_preprocess(text):
    if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
      result.append(token)

  return result

df['Resume'] = df['Resume_str'].apply(lambda w: preprocess(w))
# drop original text column
df.pop('Resume_str')
df['clean'] = df['Resume'].apply(remove_stop_words).astype(str)
df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,ID,Category,Resume,clean
0,16852973,HR,hr administrator marketing associate hr admini...,"['administrator', 'marketing', 'associate', 'a..."
1,22323967,HR,hr specialist us hr operations summary versati...,"['specialist', 'operations', 'summary', 'versa..."
2,33176873,HR,hr director summary over years experience in r...,"['director', 'summary', 'years', 'experience',..."
3,27018550,HR,hr specialist summary dedicated driven and dyn...,"['specialist', 'summary', 'dedicated', 'driven..."
4,17812897,HR,hr manager skill highlights hr skills hr depar...,"['manager', 'skill', 'highlights', 'skills', '..."
...,...,...,...,...
145,26676567,DESIGNER,graphic designer professional summary hi there...,"['graphic', 'designer', 'professional', 'summa..."
146,14724186,DESIGNER,interior designer professional summary hands o...,"['interior', 'designer', 'professional', 'summ..."
147,10748989,DESIGNER,lighting designer professional summary compute...,"['lighting', 'designer', 'professional', 'summ..."
148,26924514,DESIGNER,instructional designer professional summary an...,"['instructional', 'designer', 'professional', ..."


In [34]:
dataset = load_dataset("jacob-hugging-face/job-descriptions")

job_description = dataset['train']['job_description'][:15]
df_cv = df['Resume']

Loading DistilBERT tokenizer and model

In [35]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Tokenize and obtain embeddings for CV details and job descriptions
cv_embeddings = []
job_embeddings = []

In [36]:
for cv_text in df_cv:
    inputs = tokenizer(cv_text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    cv_embeddings.append(embeddings)

for job_text in job_description:
    inputs = tokenizer(job_text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).numpy()
    job_embeddings.append(embeddings)

In [ ]:
(np.array(cv_embeddings)).shape

(20, 1, 768)

In [ ]:
# Calculate cosine similarity between CV details and job descriptions
matching_scores = cosine_similarity(np.array(cv_embeddings).reshape(20, 768), np.array(job_embeddings).reshape(15, 768))

# Threshold for matching
threshold = 0.8

# Match job descriptions to CV details
top_matches = {}

for j, job_desc in enumerate(job_description):
  if j<15:
    scores = matching_scores[:, j]
    matched_cvs = [(i, score) for i, score in enumerate(scores) if score > threshold]
    matched_cvs.sort(key=lambda x: x[1], reverse=True)
    top_matches[j] = matched_cvs[:5]  # Get the top 5 matched CVs for each job description

# Print the matching results
for job_idx, top_cvs in top_matches.items():
    job_desc = job_description[job_idx]
    print(f"Job Description {job_idx + 1}: ")
    print("Top Matched CVs:")
    for cv_idx, score in top_cvs:
        cv_text = df_cv[cv_idx]
        print(f"- CV Detail {cv_idx + 1} (Cosine Similarity Score: {score:.5f})")
    print("-" * 50)


Job Description 1: 
Top Matched CVs:
- CV Detail 5 (Cosine Similarity Score: 0.93826)
- CV Detail 13 (Cosine Similarity Score: 0.93352)
- CV Detail 12 (Cosine Similarity Score: 0.93221)
- CV Detail 3 (Cosine Similarity Score: 0.93105)
- CV Detail 20 (Cosine Similarity Score: 0.92441)
--------------------------------------------------
Job Description 2: 
Top Matched CVs:
--------------------------------------------------
Job Description 3: 
Top Matched CVs:
- CV Detail 13 (Cosine Similarity Score: 0.94164)
- CV Detail 3 (Cosine Similarity Score: 0.94089)
- CV Detail 2 (Cosine Similarity Score: 0.94000)
- CV Detail 5 (Cosine Similarity Score: 0.93994)
- CV Detail 12 (Cosine Similarity Score: 0.93663)
--------------------------------------------------
Job Description 4: 
Top Matched CVs:
- CV Detail 3 (Cosine Similarity Score: 0.94845)
- CV Detail 5 (Cosine Similarity Score: 0.94005)
- CV Detail 18 (Cosine Similarity Score: 0.93831)
- CV Detail 11 (Cosine Similarity Score: 0.93483)
- CV D

#Using LLM

In [4]:
!pip install fastapi
!pip install openai
!pip install textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.8 MB/s eta 0:00:00


In [ ]:
import openai

# Replace with your OpenAI API key
openai.api_key = "<Enter your own API Key>"

class CVDatabase:
    def __init__(self):
        pass

    @staticmethod
    def reformat_CV(CV, fields):
        CV_info = {}
        for section in fields:
            CV_conversion_prompt = f"""
            You are a talented HR manager, your task is to EXTRACT the section mentioned in <EXTRACT SECTION> from the given Curriculum Vitae (CV) in the <CURRENT CV> section.
            Here are the rules for you:
            - WHILE EXTRACTING SECTION, DO NOT ADD ANY NEW INFORMATION, EXTRACT ONLY
            - RETURN A EMPTY STRING IF YOU DO NOT FIND RELEVANT INFORMATION TO PUT IN THE SECTION
            - RETURN ONLY THE EXTRACTED SECTION

            <CURRENT CV>
            {CV}

            <SECTION>
            {section}

            <EXTRACTED SECTION>

            """
            completion = openai.Completion.create(
                model="text-davinci-003",
                temperature=0.7,
                max_tokens=256,
                prompt=CV_conversion_prompt
            )
            extracted_section = completion.choices[0]['text']
            CV_info[section] = extracted_section.strip()
        return CV_info

# Example usage
fields = ["Summary", "Experience", "Education", "Skills", "Interests"]
sample_cv = resumes[0]

cv_database = CVDatabase()
CV_reformat = cv_database.reformat_CV(sample_cv, fields)
print(CV_reformat)